In [ ]:
# HKTVMALL 
import datetime
import math
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

# 初始化 WebDriver
driver = webdriver.Chrome()

# 瀏覽到目標頁面
driver.get("https://www.hktvmall.com/hktv/zh/search_a?keyword=%E9%A6%99%E6%B0%B4&page=0")

# 初始化 WebDriverWait
wait = WebDriverWait(driver, 5)

# 定義一個變量來跟踪產品索引
index = 1

# how many total page count , a count to help stop loop in the last page
Total_Page_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="search-result-wrapper"]/div/div[3]/div[3]/div/span'))
).text
page_TOP_num = int(re.search(r'\d+', Total_Page_element).group())

print (page_TOP_num )

# 定義一個變量來跟踪當前頁數
web_page_count = 1

# 創建一個空的 DataFrame
columns = ["Title", "Price", "Rate"]
data = pd.DataFrame(columns=columns)

while True:
    try:
        print(f"正在處理第 {web_page_count} 頁")
        # roll to the bottom to load full page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 循環處理當前頁面的所有產品
        while True:
            try:
                # 動態生成 XPath 表達式
                open_page_btn = driver.find_element(By.XPATH, f'//*[@id="algolia-search-result-container"]/div/div/span[{index}]')
                open_page_btn.click()
                driver.switch_to.window(driver.window_handles[-1])
                
                time.sleep(2)
                #close add
                try:
                    click_add = driver.find_element(By.XPATH,'/html/body/div[2]/div[6]/div/i')
                    click_add.click()
                except:
                    pass
                # 在產品頁面上提取數據
                try:
                    title = driver.find_element(By.XPATH, '//*[@id="breadcrumb"]/div[2]/ul/li[2]/h1').text
                except:
                    title = "N/A"
                try:
                    #find_brand Name
                    product_name = driver.find_elements(By.XPATH, '//*[@id="breadcrumb"]/div[2]/ul/li[2]/h1')[0].text
                    brand_name = product_name.split(" - ")[0]
                except:
                    brand_name = 'N/A'
                try:
                    price_text = driver.find_element(By.CLASS_NAME, 'price').text
                    price = price_text.split('\n')[0].strip('$')
                except:
                    price = "N/A"
                try:
                    rate = driver.find_element(By.CLASS_NAME, 'averageRating').text
                except:
                    rate = "N/A"
                # try:
                #     short_desc = driver.find_element(By.XPATH, "//span[contains(@class = 'short-desc')]").text
                # except:
                #     short_desc = "N/A"
                try:
                    origin = driver.find_elements(By.CLASS_NAME, 'productPackingSpec')[0].text
                    if '包裝' in origin:
                        origin = driver.find_elements(By.CLASS_NAME, 'productPackingSpec')[1].text
                    origin = origin.split(' ')[1]   
                except:
                    origin = "N/A"
                try: 
                    #open the graph and open 365 days
                    wait = WebDriverWait(driver, 1)
                    click_date = driver.find_element(By.XPATH, '//*[@id="pdp-graph-entrypoint-React"]/div/div[1]')
                    click_date.click()
                    click_year = driver.find_element(By.XPATH, '//*[@id="pdp-graph-entrypoint-React"]/div/div[2]/div/div[4]/div/div[5]')
                    click_year.click()
                    time.sleep(1)
                    action = webdriver.ActionChains(driver)

                    # Get the data points on the red line
                    data_points = driver.find_elements(By.XPATH, '//*[@id="pdp-graph-entrypoint-React"]/div/div[2]/div/div[3]/div[1]/div[2]/div[1]/svg/g[4]')
                    upload_date = driver.find_element(By.XPATH, '//*[@id="pdp-graph-entrypoint-React"]/div/div[2]/div/div[2]/div[1]/p[2]')
                    
                    #find the graph , detect red_line path 
                    line = driver.find_element(By.CLASS_NAME, 'recharts-cartesian-grid')
                    red_line = driver.find_element(By.CSS_SELECTOR, ".recharts-layer.recharts-line")
                    red_line_path = red_line.find_element(By.TAG_NAME, "path")
                    red_line_width = red_line_path.get_attribute("d")

                    
                    values = red_line_width.split(".")
                    Red_split = red_line_width.split("L")
                    time.sleep(1)

                    #get the red line number
                    for Red_number in Red_split:
                        if Red_number.startswith("M"):
                            Red_number_part, _ = Red_number[1:].split(",")
                            Red_number_output = math.ceil(float(Red_number_part))
                            round(Red_number_output)
                            
                    #move and point at graph get date       
                    action.move_to_element(line).move_by_offset(0, 0).perform()
                    Date_location = Red_number_output - 245

                    action.move_to_element(line).move_by_offset(Date_location, 0).perform()

                    time.sleep(1)

                    # small fix to get the date 
                    money_detect= driver.find_element(By.XPATH, '//*[@id="pdp-graph-entrypoint-React"]/div/div[2]/div/div[2]/div[2]').text
                    if "-" in money_detect  :
                        Date_location = Date_location + 1
                        action.move_to_element(line).move_by_offset(Date_location, 0).perform()

                    upload_date = upload_date.text
                except:
                    upload_date = "N/A"
                try:

                    date_month = datetime.datetime.strptime(upload_date, '%d/%m/%Y').date()
                    month = date_month.month

                    if 3 <= month <= 5:
                        season = "spring"
                    elif 6 <= month <= 8:
                        season = "summer"
                    elif 9 <= month <= 11:
                        season = "autumn"
                    else:
                        season = "winter"
                except:
                    season = "N/A"
                try :
                    today = datetime.date.today()

                    test_date_obj = datetime.datetime.strptime(upload_date, '%d/%m/%Y').date()

                    over_one_year= today - test_date_obj

                    if over_one_year.days > 364:
                        over_one_year_test = "more_then_1_year"   
                    else: 
                        over_one_year_test = "N/A"          
                except:
                    over_one_year_test = "N/A"
                time.sleep(1)
                try:
                    short_desc = "N/A"
                    if driver.find_element(By.XPATH, "//span[contains(@class, 'short-desc')]"):
                        span_element = driver.find_element(By.XPATH, "//span[contains(@class, 'short-desc')]")
                        p_elements = span_element.find_elements(By.TAG_NAME, "p")
                        p_texts = [p.text for p in p_elements]
                        short_desc = "\n".join(p_texts) 
                except:
                    short_desc = "N/A"
                    
                try:
                    full_desc = "N/A"
                    if driver.find_element(By.XPATH, '//*[@id="descriptionsTab"]/div[2]'):
                        div_element = driver.find_element(By.XPATH, '//*[@id="descriptionsTab"]/div[2]')
                        div_elements = div_element.find_elements(By.TAG_NAME, "div")
                        div_texts = [div.text for div in div_elements]
                        full_desc = "\n".join(div_texts)
                except:
                    full_desc = "N/A"

                try:
                    pattern = r"(?:前味[：:]|初味[：:]|香調[：:]|初調[：:]|前調是|[•*]?\s*前調[：:])(.*)"
                    match = re.findall(pattern, short_desc or full_desc)
                    if match:
                        front_note = match
                        front_note_str = ", ".join(front_note)
                    else:
                        front_note_str = "N/A"
                    if len(front_note_str) > 50:
                        pattern = r"初調[：:]|前味[：:]|前調[：:](.*)"
                        match = re.search(pattern, short_desc or full_desc)
                        if match:
                            front_note_str = match.group(1)
                except:
                    front_note_str = "N/A"
                    
                try :
                    pattern_mid = r"(?:中味[：:]|中調[：:]|中調是|[•*]?\s*中調[：:])(.*)"
                    match_mid = re.findall(pattern_mid, short_desc or full_desc)
                    if match_mid :
                        middle_note = match_mid 
                        middle_note_str = ", ".join(middle_note)
                    else:
                        middle_note_str  = "N/A" 
                    if len(middle_note_str) > 50:
                        pattern_mid = r"中味[：:]|中調[：:]|中調[：:](.*)"
                        match_mid  = re.search(pattern_mid, short_desc or full_desc)
                        if match_mid :
                            middle_note_str = match_mid.group(1)
                except:
                    middle_note_str  = "N/A"    

                try :
                    pattern_base = r"(?:後味[：:]|後味[：:]|後調[：:]|基調[：:]|後味是|後調是|[•*]?\s*後調[：:])(.*)"
                    match_base = re.findall(pattern_base, short_desc or full_desc)
                    if match_base:
                        base_note = match_base
                        base_note_str = ", ".join(base_note)
                    else:
                        base_note_str  = "N/A"
                    if len(base_note_str) > 50:
                        pattern_base = r"中味[：:]|中調[：:]|中調[：:](.*)"
                        match_base = re.search(pattern_base, short_desc or full_desc)
                        if match_base:
                            base_note_str = match_base.group(1)
                except:
                    base_note_str  = "N/A"
                    
                    
                    
                # 將數據添加到 DataFrame
                new_row = pd.DataFrame({"Title": [title],"brand_name":[brand_name],"front_note":[front_note_str],"middle_note":[middle_note_str],"base_note":[base_note_str]
                                        , "Price": [price], "Rate": [rate], "Origin": [origin]
                                        , "upload_date": [upload_date], "season": [season], "over_one_year_test": [over_one_year_test]})
                data = pd.concat([data, new_row], ignore_index=True)

                # 關閉新窗口並切換回主窗口
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                
                # 更新索引以定位下一個產品
                index += 1
                
            except Exception as e:
                # 如果出現異常，說明沒有更多的產品鏈接
                print(f"第 {web_page_count} 頁已經處理完所有產品或出現錯誤：", e)
                break

        # if no more page to click , BREAK
        web_page_count = web_page_count + 1
        print (f'{web_page_count}____________')
        print (page_TOP_num)
        # if get up to the max number of page , BREAK
        if web_page_count > page_TOP_num:
            break
        # 重置產品索引
        index = 1

        # 嘗試找到下一頁按鈕並檢查是否可點擊
        try:
            nextPageButton = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@id, 'paginationMenu_nextBtn')]")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            nextPageButton.click()
            time.sleep(3)
            print ("Next _page _GO! ")
            
        except:
            print("已經到達最後一頁")
            break

    except Exception as e:
        print(f"已經處理完所有頁面或出現錯誤：", e)
        break

# 完成後關閉瀏覽器
driver.quit()

# 輸出 DataFrame
print(data)

#display(data) 

#data.to_csv('HKTVMALL_DATE_TEXT.csv', index=False)


In [ ]:
# FOR Fragrance
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Initialize Chrome WebDriver
driver = webdriver.Chrome()
url = "https://fragrance.com.hk/"
driver.get(url)
time.sleep(1)

# Input search keyword
input_keyword = '香水'
search_box_compo = driver.find_element(By.XPATH, '//*[@id="woocommerce-product-search-field-1"]')
search_box_compo.send_keys(input_keyword)

# Click search button
search_button_compo = driver.find_element(By.XPATH, '//*[@id="site-header-inner"]/div/section[3]/div/div[2]/div/div/div/div/form/button')
search_button_compo.click()
time.sleep(5)

# Function to collect all product links
def collect_product_links():
    product_links = []
    while True:
        result_list = driver.find_elements(By.XPATH, "//div[contains(@class,'product-inner clr')]")
        for result in result_list:
            product_link = result.find_element(By.TAG_NAME, 'a').get_attribute('href')
            product_links.append(product_link)
        try:
            next_button = driver.find_element(By.XPATH, "//a[@class='next page-numbers']")
            next_button.click()
            time.sleep(5)
        except:
            break
    return product_links

# Collect all product links
product_links = collect_product_links()

# Initialize lists to store data
special_offer = []
title_list = []
price1_list = []
price2_list = []
top_notes = []
heart_notes = []
base_notes = []

# Function to scrape product details from each link
def scrape_product_details(link):
    driver.get(link)
    time.sleep(3)
    
    try:
        # Special offer
        special_offer_elem = driver.find_element(By.XPATH, '//span[@class="onsale"]')
        special_offer.append(special_offer_elem.text)
    except Exception as e:
        special_offer.append(None)
    
    # Title
    title_elem = driver.find_element(By.XPATH, '//h1[@class="product_title entry-title"]')
    title_list.append(title_elem.text)
    
    # Price 1
    price1_elem = driver.find_element(By.XPATH, '//p[@class="price"]//ins/span')
    price1_list.append(price1_elem.text.replace("$", ""))
    
    # Price 2 (if available)
    try:
        price2_elem = driver.find_element(By.XPATH, '//p[@class="price"]//del/span')
        price2_list.append(price2_elem.text.replace("$", ""))
    except Exception as e:
        price2_list.append(None)
    
    # Fragrance notes
    try:
        tab_description = driver.find_element(By.XPATH, '//*[@id="tab-description"]')
        notes_text = tab_description.text
        
        # Extract notes
        notes = notes_text.split('\n')
        top_note = None
        heart_note = None
        base_note = None
        
        for line in notes:
            if any(marker in line for marker in ['前調: ', '前調:', '前調：', '香調：', '"前調:', 'M前調：', '調：', '"前調: ',
                                                 '"前調：', '香味: ', '香氣: ', '前調： ', '"前調： ', '"香味: ', '"香味：',
                                                 '香味：', 'Top Notes: ', '"Middle Notes: ', '香調：木質花香調,前調：']):
                top_note = line.split(':')[-1].strip()
            elif any(marker in line for marker in ['中調： ', '中調：', '中調:', '中調: ', '"中調: ', '"Middle Notes: ', 'Middle Notes: ',
                                                   '中調:  ', '"heart note: ', '"Scent: ', '"中調：', '"scent:', '"Scent ',
                                                   '"heart note:', '"中調： ', '"中調:', '中調：  ']):
                heart_note = line.split(':')[-1].strip()
            elif any(marker in line for marker in ['基調： ', '基調：', '基調: ', '基調:', '基調:   ', '基調:  ', '"基調：', '"基調： ',
                                                   '"Base note: ', '"基調: ', '"Base Notes:  ', '後調：', '基調：   ', '"基調：',
                                                   '"基調:', '"Base Notes:', 'Base Notes: ', '"後調: ','後調:']):
                base_note = line.split(':')[-1].strip()
        
        top_notes.append(top_note)
        heart_notes.append(heart_note)
        base_notes.append(base_note)
    except Exception as e:
        top_notes.append(None)
        heart_notes.append(None)
        base_notes.append(None)

# Scrape details from each product link
for link in product_links:
    try:
        scrape_product_details(link)
    except Exception as e:
        print(f"Error processing {link}: {str(e)}")

# Ensure all lists have the same length
min_length = min(len(special_offer), len(title_list), len(price1_list), len(price2_list), len(top_notes), len(heart_notes), len(base_notes))
special_offer = special_offer[:min_length]
title_list = title_list[:min_length]
price1_list = price1_list[:min_length]
price2_list = price2_list[:min_length]
top_notes = top_notes[:min_length]
heart_notes = heart_notes[:min_length]
base_notes = base_notes[:min_length]


# Create DataFrame
df = pd.DataFrame({
    'Special_offer': special_offer,
    'Title': title_list,
    'Price1': price1_list,
    'Price2': price2_list,
    'Top_notes': top_notes,
    'Heart_notes': heart_notes,
    'Base_notes': base_notes
})

# Display the DataFrame
display(df)

















In [ ]:
# FOR Sasa product

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException
import pandas as pd
import schedule
import time
from datetime import datetime

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def accept_cookies(driver):
    try:
        iknow_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[7]/a')))
        iknow_button.click()
    except TimeoutException:
        print("未找到接受 Cookies 的按鈕")

def close_ad(driver):
    try:
        close_ad_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[7]')))
        close_ad_button.click()
    except TimeoutException:
        print("未找到關閉廣告的按鈕")

def navigate_to_category(driver, category_name):
    try:
        category_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//div[text()="{category_name}"]')))
        category_button.click()
    except TimeoutException:
        print(f"未找到分類為 {category_name} 的按鈕")
        driver.quit()

def scroll_to_bottom(driver):
    initial_result_count = len(driver.find_elements(By.XPATH, '//div[@class="sc-cOFCrh bNMPZm"]'))
    result_unchanged_count = 0

    while result_unchanged_count < 10:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            WebDriverWait(driver, 5).until(EC.staleness_of(driver.find_element(By.XPATH, '//div[@class="sc-cOFCrh bNMPZm"]')))
        except TimeoutException:
            pass
        
        current_result_count = len(driver.find_elements(By.XPATH, '//div[@class="sc-cOFCrh bNMPZm"]'))
        if current_result_count == initial_result_count:
            result_unchanged_count += 1
        else:
            result_unchanged_count = 0
        initial_result_count = current_result_count

def extract_product_data(driver, link):
    # 提取產品信息
    try:
        driver.get(link)
        wait = WebDriverWait(driver, 20)  # 增加等待時間至 20 秒
        product_name = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'salepage-title'))).text
        brand_name = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'salepage-brand-list'))).text
        
        special_price_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@class="cms-discountMoneyColor" and contains(text(), "HK$")]')))
        special_price_text = special_price_element.text.replace("HK$", "").strip()
        
        original_price_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="salepage-suggestprice" and contains(text(), "HK$")]')))
        original_price_text = original_price_element.text.replace("HK$", "").strip()
        
        a_tags = driver.find_elements(By.XPATH, '//a[@class="sku-link cms-primaryBtnTextColor"]')
        capacity_text = a_tags[0].text.strip() if a_tags else None

        # 提取成分信息
        ingredient_section = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="salepage-detail-info"]/div[2]')))
        ingredients = ingredient_section.text

        # 初始化前調、中調、基調
        top_notes = None
        heart_notes = None
        base_notes = None

        # 找到前調信息
        top_keywords = ["• 前調：", "• 前調:", "前調：", "前調:"]
        top_start_index = -1
        for keyword in top_keywords:
            if keyword in ingredients:
                top_start_index = ingredients.find(keyword) + len(keyword)
                break

        # 找到中調信息
        heart_keywords = ["• 中調：", "• 中調:", "中調：", "中調:"]
        heart_start_index = -1
        for keyword in heart_keywords:
            if keyword in ingredients:
                heart_start_index = ingredients.find(keyword) + len(keyword)
                break

        # 找到基調信息
        base_keywords = ["• 基調：", "• 基調:", "• 後調：", "• 後調:", "後調:", "後調：", "基調：", "基調:"]
        base_start_index = -1
        for keyword in base_keywords:
            if keyword in ingredients:
                base_start_index = ingredients.find(keyword) + len(keyword)
                break

        # 提取前調信息
        if top_start_index != -1:
            next_note_index = min([i for i in [heart_start_index, base_start_index] if i != -1], default=len(ingredients))
            top_notes = ingredients[top_start_index:next_note_index].strip().split("\n")[0]

        # 提取中調信息
        if heart_start_index != -1:
            next_note_index = min([i for i in [base_start_index] if i != -1], default=len(ingredients))
            heart_notes = ingredients[heart_start_index:next_note_index].strip().split("\n")[0]

        # 提取基調信息
        if base_start_index != -1:
            base_notes = ingredients[base_start_index:].strip().split("\n")[0]

        return {
            'product_name': product_name,
            'Original_Price': original_price_text,
            'Special_Price': special_price_text,
            'capacity': capacity_text,
            'brand': brand_name,
            'top_notes': top_notes,
            'heart_notes': heart_notes,
            'base_notes': base_notes
        }
    
    except (NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException) as e:
        print(e)
        return {
            'product_name': None,
            'Original_Price': None,
            'Special_Price': None,
            'capacity': None,
            'brand': None,
            'top_notes': None,
            'heart_notes': None,
            'base_notes': None
        }

def main():
    driver = initialize_driver()
    try:
        url = "https://www.sasa.com.hk/"
        driver.get(url)
        accept_cookies(driver)
        close_ad(driver)
        navigate_to_category(driver, "商品分類")
        navigate_to_category(driver, "香水香薰")
        scroll_to_bottom(driver)

        product_links = []
        result_list = driver.find_elements(By.XPATH, '//li[@class="column-grid-container__column"]')
        for result in result_list:
            a_tag = result.find_element(By.TAG_NAME, 'a')
            link = a_tag.get_attribute('href')
            product_links.append(link)

        extracted_data = []
        for link in product_links:
            data = extract_product_data(driver, link)
            if data:
                extracted_data.append(data)

        df = pd.DataFrame(extracted_data)
        timestamp = datetime.now().strftime('%Y%m%d')
        filename = f'sasa_flavor_{timestamp}.csv'
        df.to_csv(filename, index=False)
        print(f"數據提取完成，CSV 文件保存為 {filename}")
        
    finally:
        driver.quit()

def run_crawler():
    schedule.every().day.at("00:33").do(main)

    while True:
        schedule.run_pending()
        time.sleep(60)  # 每隔一分鐘檢查一次是否有任務需要運行

if __name__ == "__main__":
    run_crawler()

In [ ]:
#sasa product comment (from 香水時代評價)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
import difflib
import opencc 

# Load CSV file
csv_file_path = r'C:\Users\nat chan\OneDrive\桌面\JDE10A\Mid-Project\clean_product_names.csv'
df = pd.read_csv(csv_file_path)

# Extract product names from CSV
product_names = df['product_name'].tolist()

# Initialize WebDriver
options = webdriver.ChromeOptions()
# Uncomment the line below for headless mode
# options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
url = "https://www.nosetime.com/"
driver.get(url)
time.sleep(5)

# Function to handle potential blocking elements
def close_blocking_elements(driver):
    try:
        close_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[ng-click="cancel()"]'))
        )
        close_button.click()
        print("Blocking element closed successfully.")
        time.sleep(2)  # Wait for page to update
    except TimeoutException:
        print("No blocking element found or unable to click.")

sasa_product_name_list = []
sasa_comment_rate_list = []
number_of_raters_list = []

# Initialize OpenCC for traditional to simplified Chinese conversion
cc = opencc.OpenCC('t2s')

# Iterate through all product names for search
for index, input_text in enumerate(product_names):
    try:
        # Ensure input_text is a string
        if not isinstance(input_text, str):
            input_text = str(input_text)

        # Convert traditional Chinese to simplified Chinese
        input_text_simplified = cc.convert(input_text)

        # Locate the search box and input search text
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input.input'))
        )
        search_box.clear()
        search_box.send_keys(input_text_simplified)

        # Click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[2]/form/input[1]'))
        )
        search_button.click()

        # Handle potential blocking elements
        close_blocking_elements(driver)

        # Wait for search results to load
        links_list = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[2]/ul/div/li[1]/div/div/h2/a'))
        )

        if links_list:
            comment_link = links_list[0].get_attribute('href')
            driver.get(comment_link)
            time.sleep(2)

            # Handle potential blocking elements again
            close_blocking_elements(driver)

            # Extract product name
            sasa_product = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/ul[1]/h1'))
            ).text.strip()
            print(f"Found product name: {sasa_product}")

            # Convert traditional Chinese to simplified Chinese for comparison
            sasa_product_simplified = cc.convert(sasa_product)

            # Check if found product is similar to input_text
            similarity = difflib.SequenceMatcher(None, input_text_simplified.lower(), sasa_product_simplified.lower()).ratio()
            if similarity > 0.5:  # Adjusted threshold to 0.5
                # Extract rating (if available)
                try:
                    sasa_comment_rate_element = WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located((By.XPATH, '//span[contains(@class, "star") and contains(@class, "sa")]'))
                    )
                    class_attribute = sasa_comment_rate_element.get_attribute('class')
                    rating = int(class_attribute.split('sa')[-1]) / 2
                    sasa_comment_rate = f"{rating:.1f}"
                    print(f"Found rating: {sasa_comment_rate}")
                except TimeoutException:
                    sasa_comment_rate = None
                    print("Rating not found.")

                # Extract number of raters
                try:
                    number_of_raters_element = WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located((By.CLASS_NAME, 'people'))
                    )
                    number_of_raters = number_of_raters_element.text.replace(" 人评分", "")
                    print(f"Found number of raters: {number_of_raters}")
                except TimeoutException:
                    print("Primary number of raters element not found, attempting secondary path.")
                    try:
                        number_of_rater_element = WebDriverWait(driver, 10).until(
                            EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/ul[1]/ul[2]/li/div[1]/div[2]/span[2]'))
                        )
                        number_of_raters = number_of_rater_element.text.replace(" 人评分", "")
                        print(f"Found number of raters (secondary path): {number_of_raters}")
                    except TimeoutException:
                        number_of_raters = None
                        print("Number of raters not found.")
            else:
                print(f"Product name mismatch: expected '{input_text}', found '{sasa_product}'")
                sasa_product = None
                sasa_comment_rate = None
                number_of_raters = None
        else:
            print(f"No search results found for '{input_text}'")
            sasa_product = None
            sasa_comment_rate = None
            number_of_raters = None

    except (TimeoutException, NoSuchElementException) as e:
        print(f"Error processing '{input_text}': {e}")
        sasa_product = None
        sasa_comment_rate = None
        number_of_raters = None

    # Append results to lists
    sasa_product_name_list.append(sasa_product)
    sasa_comment_rate_list.append(sasa_comment_rate)
    number_of_raters_list.append(number_of_raters)

    # Return to main page
    driver.get(url)
    time.sleep(2)

# Close the browser
driver.quit()

# Construct DataFrame from collected data
df_result = pd.DataFrame({
    'Product Name': product_names[:len(sasa_product_name_list)],
    'Sasa Product Name': sasa_product_name_list,
    'Comment Rate': sasa_comment_rate_list,
    'Number of Raters': number_of_raters_list
})
# Display DataFrame
display(df_result)